# Pharmacy Department - Why KPI is not met

In this section, we will look at why certain cases do not meet KPI. 

This can due to the process time taken to complete each case or if a particular ward is overloading etc.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("../../Data/Pharmacy Dept/Data Cleaning/Merged_Pharmacy_Dept_CLEANED.csv")

df.head()

In [ ]:
df.info()

## KPI Analysis

Extracting columns that are needed to calculate the percentage of KPI met per day and per month.

In [ ]:
#Extract those that needed only
df_kpi = df[['Date', 'Ward', 'Room/Bed', 'Case Number', 'Hospital', 'No. of Drugs', 'Weekday', 'Office Hours', 'TAT', 'Meet KPI', 'Process Time Issue', 'Time Taken (Received to Checked)', 'Time Taken (Checked to Dispensed)', 'Avg Time Taken / Day (Received to Checked)', 'Avg Time Taken / Day (Checked to Dispensed)', 'Avg Time Taken / Month (Received to Checked)', 'Avg Time Taken / Month (Checked to Dispensed)']]
df_kpi

In [ ]:
df_kpi[df_kpi['Meet KPI'].isnull()]

### Percentage of KPI met / Day

Understanding the percentage of KPI that are met per day. 
Taking into assumption of ignoring outliers. 

In [ ]:
#create new dataframe to calculate no. of KPI / day
df_kpi_sum = df_kpi[['Date', 'Hospital', 'Meet KPI']]
df_kpi_sum.head()

In [ ]:
df_kpi_sum.isnull().sum()

In [ ]:
df_kpi_sum[df_kpi_sum['Meet KPI'].isnull()]

In [ ]:
#convert Meet KPI (Yes/No) to 1/0
df_kpi_sum['Meet KPI'] = df_kpi_sum['Meet KPI'].replace(to_replace=['Yes', 'No'], value = ['1', '0'])
df_kpi_sum

In [ ]:
#drop Meet KPI = NaN
df_kpi_sum = df_kpi_sum.dropna()

In [ ]:
df_kpi_sum.isnull().sum()

In [ ]:
#set Meet KPI type as int
df_kpi_sum['Meet KPI'] = df_kpi_sum['Meet KPI'].astype(str).astype(int)

In [ ]:
#Get No. of Cases that Meet KPI per day
df_kpi_sum_day = df_kpi_sum.groupby(['Hospital', pd.to_datetime(df_kpi_sum['Date']).dt.date]).sum()
df_kpi_sum_day

In [ ]:
#create new dataframe to calculate No. of Cases / Day
df_case_sum = df_kpi[['Date', 'Hospital', 'Meet KPI']]
df_case_sum

In [ ]:
df_case_sum.isnull().sum()

In [ ]:
df_case_sum = df_case_sum[df_case_sum['Meet KPI'].notnull()]
df_case_sum['Case Count'] = 1
df_case_sum

In [ ]:
df_case_sum.drop(['Meet KPI'], axis=1, inplace=True)

In [ ]:
#Get No. of Cases Per Day
df_case_sum_day = df_case_sum.groupby(['Hospital', pd.to_datetime(df_case_sum['Date']).dt.date]).sum()
df_case_sum_day

In [ ]:
#merge both No. of Cases that meet KPI and No. Cases / Day
df_kpi_percent_day = pd.merge(df_kpi_sum_day, df_case_sum_day, on=['Hospital', 'Date'])
df_kpi_percent_day

In [ ]:
df_kpi_percent_day['%KPI / Day'] = df_kpi_percent_day['Meet KPI']/df_kpi_percent_day['Case Count']
df_kpi_percent_day

In [ ]:
#if 85% meet KPI / day: Yes, Else No
df_kpi_percent_day['85% Meet KPI / Day'] = np.where((df_kpi_percent_day['%KPI / Day'] >= 0.85), 'Yes', 'No')
df_kpi_percent_day

### Percentage of KPI met / Month

Understanding the percentage of KPI that are met per month. Taking into assumption of ignoring outliers. 

In [ ]:
#Get No. of KPI met Per Month
df_kpi_sum_mth = df_kpi_sum.groupby(['Hospital', pd.to_datetime(df_kpi_sum['Date']).dt.month]).sum()
df_kpi_sum_mth

In [ ]:
#Get No. of Cases Per Month
df_case_sum_mth = df_case_sum.groupby(['Hospital', pd.to_datetime(df_case_sum['Date']).dt.month]).sum()
df_case_sum_mth

In [ ]:
#merge both No. of Cases that meet KPI and No. Cases / Month
df_kpi_percent_mth = pd.merge(df_kpi_sum_mth, df_case_sum_mth, on=['Hospital', 'Date'])
df_kpi_percent_mth

In [ ]:
df_kpi_percent_mth['%KPI / Mth'] = df_kpi_percent_mth['Meet KPI']/df_kpi_percent_mth['Case Count']
df_kpi_percent_mth

In [ ]:
#if 85% meet KPI / day: Yes, Else No
df_kpi_percent_mth['85% Meet KPI / Mth'] = np.where((df_kpi_percent_mth['%KPI / Mth'] >= 0.85), 'Yes', 'No')
df_kpi_percent_mth

### Understanding the difference between KPI met / Day and / Month

#### Merge KPI met /  Day and / Month

In [ ]:
#reset index
df_kpi_percent_day = df_kpi_percent_day.reset_index()
df_kpi_percent_day.head()

In [ ]:
#reset index
df_kpi_percent_mth = df_kpi_percent_mth.reset_index()
df_kpi_percent_mth.head()

In [ ]:
#add Month column to both df for easy merge
df_kpi_percent_day['Month'] = pd.to_datetime(df_kpi_percent_day['Date']).dt.month_name()
df_kpi_percent_mth['Month'] = pd.to_datetime(df_kpi_percent_mth['Date'], format='%m').dt.month_name()

In [ ]:
#rename variables for merge
df_kpi_percent_day.rename(columns={'Meet KPI':'Meet KPI / Day', 'Case Count':'Case Count / Day'}, inplace=True)
df_kpi_percent_mth.rename(columns={'Meet KPI':'Meet KPI / Mth', 'Case Count':'Case Count / Mth'}, inplace=True)

In [ ]:
df_kpi_percent_day.head()

In [ ]:
df_kpi_percent_mth.head()

In [ ]:
dfinal_kpi_percent = pd.merge(df_kpi_percent_day, df_kpi_percent_mth, on=['Hospital', 'Month'], how='left')
dfinal_kpi_percent

In [ ]:
#drop Date_y, Month
dfinal_kpi_percent.drop(['Month', 'Date_y'], axis=1, inplace=True)
#rename Date_x to Date
dfinal_kpi_percent.rename(columns={'Date_x':"Date"}, inplace=True)

In [ ]:
dfinal_kpi_percent

#### Merge ALL KPI new indicators with Original KPI df

In [ ]:
#convert Date to DateTime format
df_kpi['Date'] = pd.to_datetime(df_kpi['Date'], format='%Y-%m-%d').dt.date
dfinal_kpi_percent['Date'] = pd.to_datetime(dfinal_kpi_percent['Date'], format='%Y-%m-%d').dt.date

In [ ]:
dfinal_kpi = pd.merge(df_kpi, dfinal_kpi_percent, on=['Date', 'Hospital'], how='left')
dfinal_kpi

#### Extracting KPI == NO for Power BI evaluation

In [ ]:
dfinal_kpi_NO = dfinal_kpi[dfinal_kpi['Meet KPI']=='No']

In [ ]:
dfinal_kpi_NO.info()

## Export to CSV

In [ ]:
dfinal_kpi_NO.to_csv("../../Data/Pharmacy Dept/Why_KPI_Not_Met.csv", index=None, header=True)